In [3]:
from anytree import Node, RenderTree
import json
from pymongo import MongoClient
import glob
import json

In [5]:
client = MongoClient('mongodb://localhost:27017/')

db = client.get_database('tweets')

In [10]:
# get time difference of parent tweet and child tweet
from datetime import datetime
def difference_datetime(child_datetime, parent_datetime):
    first_datetime = datetime.strftime(datetime.strptime(child_datetime,'%a %b %d %H:%M:%S +0000 %Y'), '%Y-%m-%d %H:%M:%S')
    first_datetime = datetime.strptime(first_datetime, '%Y-%m-%d %H:%M:%S')
    later_datetime = datetime.strftime(datetime.strptime(parent_datetime,'%a %b %d %H:%M:%S +0000 %Y'), '%Y-%m-%d %H:%M:%S')
    later_datetime = datetime.strptime(later_datetime, '%Y-%m-%d %H:%M:%S')
    return first_datetime - later_datetime

In [6]:
tweets = db['data'].find({})
followers = db['followers'].find({})

tweet_user_ids = []
for tweet in tweets:
    tweet_user_ids.append(tweet['user']['id'])

In [135]:
# import tree
from anytree.importer import DictImporter
from anytree import RenderTree
importer = DictImporter()
with open('tree.json', 'r') as fp:
    data = fp.readline()
data = json.loads(data)
root = importer.import_(data)
print(RenderTree(root))

AnyNode(created_at='Tue Mar 30 00:52:05 +0000 2021', difference_seconds=0, name=621574143)
├── AnyNode(created_at='Tue Mar 30 15:28:45 +0000 2021', difference_seconds=52600.0, name=26116609)
├── AnyNode(created_at='Tue Mar 30 13:31:14 +0000 2021', difference_seconds=45549.0, name=45837833)
├── AnyNode(created_at='Wed Mar 31 07:17:17 +0000 2021', difference_seconds=109512.0, name=2800570890)
├── AnyNode(created_at='Tue Mar 30 21:00:46 +0000 2021', difference_seconds=72521.0, name=1149954571)
├── AnyNode(created_at='Tue Mar 30 21:08:55 +0000 2021', difference_seconds=73010.0, name=742343184)
├── AnyNode(created_at='Wed Mar 31 11:21:48 +0000 2021', difference_seconds=124183.0, name=229263888)
├── AnyNode(created_at='Tue Mar 30 13:15:30 +0000 2021', difference_seconds=44605.0, name=260981779)
├── AnyNode(created_at='Tue Mar 30 20:16:51 +0000 2021', difference_seconds=69886.0, name=602944020)
├── AnyNode(created_at='Tue Mar 30 23:29:58 +0000 2021', difference_seconds=81473.0, name=143974402

In [130]:
# Add depth two nodes to the cascade
for child in root.children:
    followers = db['followers'].find({'user_id': child.name})
    follower = list(followers)
    if not len(follower) == 0:
        child_followers = follower[0]['followers'] 
        child_potential_user_ids = list(set(child_followers) & set(tweet_user_ids))
        # add depth 2 users to the tree and tweet created time
        for user_id in child_potential_user_ids:
            tweets = db['data'].find({'user.id':{'$in': child_potential_user_ids}})
            for tweet in tweets:
                if tweet['user']['id'] == user_id:
                    difference = difference_datetime(tweet['created_at'], child.created_at).total_seconds()
                    if difference >= 0:
                        globals()[f'user_{user_id}'] = Node(name = user_id, created_at = tweet['created_at'], difference_seconds = difference , parent = child)

for pre, fill, node in RenderTree(root):
    print("%s user_id: %s, diff_s: %s" % (pre, node.name, node.difference_seconds))

 user_id: 621574143, diff_s: 0
├──  user_id: 26116609, diff_s: 52600.0
├──  user_id: 45837833, diff_s: 45549.0
├──  user_id: 2800570890, diff_s: 109512.0
├──  user_id: 1149954571, diff_s: 72521.0
├──  user_id: 742343184, diff_s: 73010.0
├──  user_id: 229263888, diff_s: 124183.0
├──  user_id: 260981779, diff_s: 44605.0
├──  user_id: 602944020, diff_s: 69886.0
├──  user_id: 1439744029, diff_s: 81473.0
├──  user_id: 1547202078, diff_s: 86758.0
├──  user_id: 2808650785, diff_s: 49802.0
│   ├──  user_id: 33622022, diff_s: 1217.0
│   ├──  user_id: 480104454, diff_s: 80847.0
│   ├──  user_id: 940185098, diff_s: 12107.0
│   ├──  user_id: 18369042, diff_s: 41244.0
│   ├──  user_id: 63263764, diff_s: 40799.0
│   ├──  user_id: 17701397, diff_s: 51245.0
│   ├──  user_id: 270458388, diff_s: 89010.0
│   ├──  user_id: 1453177374, diff_s: 616.0
│   ├──  user_id: 31703582, diff_s: 145803.0
│   ├──  user_id: 45638688, diff_s: 24210.0
│   ├──  user_id: 42408999, diff_s: 32303.0
│   ├──  user_id: 26135760

In [131]:
# remove redundant nodes
# if there are redundant user_ids keep only the most loweres diff_s

from anytree import Node, RenderTree, AsciiStyle, PreOrderIter

#get the length of the node
a = [node.length for node in PreOrderIter(root)]
print(a)

AttributeError: 'AnyNode' object has no attribute 'length'

In [128]:
names = []
for i in range(len(a)):
    depth_of_node = len(a[i])
    names.append(a[i][depth_of_node-1].name)

TypeError: object of type 'AnyNode' has no len()

In [119]:

# find duplicates
import collections
from operator import attrgetter

duplicates = [item for item, count in collections.Counter(names).items() if count > 1]
print(len(duplicates))
import anytree

all_nodes_tobe_remove = []
for duplicate in duplicates:
#     print(duplicate)
    nodes = anytree.search.findall_by_attr(root, duplicate)
#     print(nodes)
    node_created_at = set()
    node_different_seconds = []
    # if not crated_at is equal
        # keep the olderst node
    # else
        # keep the closest node to the parent
                
    # get closest node to the parent
    difference_seconds_sorted = sorted(nodes, key=lambda x: x.difference_seconds)
    
    for node in nodes:
        node_created_at.add(node.created_at)
        node_different_seconds.append(node.difference_seconds)
        
    if not len(node_created_at) ==  1:
        print('keep the olderst node', difference_seconds_sorted[0])
    else:
#         print('keep the closest node to the parent: ', min(node_different_seconds), difference_seconds_sorted[0])
        all_nodes_tobe_remove.extend(difference_seconds_sorted[1:])
print(len(all_nodes_tobe_remove))

100
keep the olderst node AnyNode(created_at='Tue Mar 30 00:52:05 +0000 2021', difference_seconds=0, name=621574143)
181


In [122]:
for node in all_nodes_tobe_remove[:5]:
    del node

In [124]:
for pre, fill, node in RenderTree(root):
    print("%s user_id: %s, diff_s: %s" % (pre, node.name, node.difference_seconds))

 user_id: 621574143, diff_s: 0
├──  user_id: 26116609, diff_s: 52600.0
├──  user_id: 45837833, diff_s: 45549.0
├──  user_id: 2800570890, diff_s: 109512.0
├──  user_id: 1149954571, diff_s: 72521.0
├──  user_id: 742343184, diff_s: 73010.0
├──  user_id: 229263888, diff_s: 124183.0
├──  user_id: 260981779, diff_s: 44605.0
├──  user_id: 602944020, diff_s: 69886.0
├──  user_id: 1439744029, diff_s: 81473.0
├──  user_id: 1547202078, diff_s: 86758.0
├──  user_id: 2808650785, diff_s: 49802.0
│   ├──  user_id: 33622022, diff_s: 1217.0
│   ├──  user_id: 480104454, diff_s: 80847.0
│   ├──  user_id: 940185098, diff_s: 12107.0
│   ├──  user_id: 18369042, diff_s: 41244.0
│   ├──  user_id: 63263764, diff_s: 40799.0
│   ├──  user_id: 17701397, diff_s: 51245.0
│   ├──  user_id: 270458388, diff_s: 89010.0
│   ├──  user_id: 1453177374, diff_s: 616.0
│   ├──  user_id: 31703582, diff_s: 145803.0
│   ├──  user_id: 45638688, diff_s: 24210.0
│   ├──  user_id: 42408999, diff_s: 32303.0
│   ├──  user_id: 26135760

In [103]:
>>> n = Node("n")
>>> a = Node("a", parent=n)
>>> b = Node("b", parent=n)
>>> c = Node("c", parent=n)
>>> d = Node("d")
>>> n.children

(Node('/n/a'), Node('/n/b'), Node('/n/c'))

In [110]:
>>> n.children = [a,d]

In [111]:
>>> n.children

(Node('/n/a'), Node('/n/d'))